#Task One: Aspect_Category NER

In [4]:
!pip install datasets evaluate transformers[sentencepiece] --quiet
!pip install seqeval --quiet

In [5]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [6]:
import pandas as pd
data = pd.read_excel('/kaggle/input/dataset1/All_data.xlsx')

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [7]:
data.head()

,Sentence #,word,aspect,categort,polarity
0,Sentence:0,عمال,O,O,O
1,Sentence:0,شركات,O,O,O
2,Sentence:0,التوصيل,O,O,O
3,Sentence:0,أمثال,O,O,O
4,Sentence:0,ذبحونا,B,Service,Neg


In [8]:
data['aspect_category'] = data['aspect'] + '-' + data['categort']
data.drop(['aspect', 'categort'], axis=1, inplace=True)

In [9]:
data['aspect_category'] = data['aspect_category'].replace(['O-O', 'B-O'], 'O')
data.head()

,Sentence #,word,polarity,aspect_category
0,Sentence:0,عمال,O,O
1,Sentence:0,شركات,O,O
2,Sentence:0,التوصيل,O,O
3,Sentence:0,أمثال,O,O
4,Sentence:0,ذبحونا,Neg,B-Service


In [10]:
aspects = ["Service", "Price", "Time", "Location"]
data = data[data['aspect_category'].str.contains('|'.join(aspects)) | (data['aspect_category'] == 'O')]

In [11]:
data.head()

,Sentence #,word,polarity,aspect_category
0,Sentence:0,عمال,O,O
1,Sentence:0,شركات,O,O
2,Sentence:0,التوصيل,O,O
3,Sentence:0,أمثال,O,O
4,Sentence:0,ذبحونا,Neg,B-Service


In [12]:
from datasets import Dataset, DatasetDict, Features, Sequence, ClassLabel, Value
import random

dataset_task_one = []
label_set = set()

for i, (sent_id, group) in enumerate(data.groupby("Sentence #")):
    tokens = group["word"].tolist()
    tags = group["aspect_category"].tolist()
    label_set.update(tags)
    dataset_task_one.append({
        "id": str(i),
        "tokens": tokens,
        "ner_tags": tags
    })

label_list = sorted(label_set)
label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

for item in dataset_task_one:
    item["ner_tags"] = [label2id[tag] for tag in item["ner_tags"]]

features = Features({
    "id": Value("string"),
    "tokens": Sequence(Value("string")),
    "ner_tags": Sequence(ClassLabel(names=label_list))
})


In [13]:
n = len(dataset_task_one)
train_size = int(0.8 * n)
val_size = int(0.05 * n)
test_size = n - train_size - val_size

data = DatasetDict({
    "train": Dataset.from_list(dataset_task_one[:train_size], features=features),
    "validation": Dataset.from_list(dataset_task_one[train_size:train_size + val_size], features=features),
    "test": Dataset.from_list(dataset_task_one[train_size + val_size:], features=features)
})

label_names = label_list

In [14]:
from transformers import AutoTokenizer

model_checkpoint = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [15]:
inputs = tokenizer(data['train'][0]['tokens'], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Ø¹',
 'Ùħ',
 'Ø§ÙĦ',
 'Ø´',
 'Ø±',
 'Ùĥ',
 'Ø§Øª',
 'Ø§ÙĦ',
 'Øª',
 'ÙĪ',
 'Øµ',
 'ÙĬ',
 'ÙĦ',
 'Ø£',
 'Ùħ',
 'Ø«',
 'Ø§ÙĦ',
 'Ø°',
 'Ø¨',
 'ØŃ',
 'ÙĪÙĨ',
 'Ø§',
 'Ø§Ø¨',
 'Ø¹',
 'Ø«',
 'ÙĦ',
 'ÙĬ',
 'ÙĦ',
 'ÙĪ',
 'Ùĥ',
 'ÙĬ',
 'Ø´',
 'ÙĨ',
 'Ùģ',
 'ÙĬ',
 'Ùĥ',
 'ÙĦ',
 'Ùħ',
 'Ø±',
 'Ø©',
 'Øª',
 'ÙĪ',
 'Øµ',
 'ÙĬ',
 'ÙĦ',
 'Ùħ',
 'Ø«',
 'ÙĦ',
 'Ùħ',
 'Ø§',
 'Ø®',
 'Ø²',
 'ÙĨ',
 'Øª',
 'Ùħ',
 'Øª',
 'ÙĦ',
 'ÙĬ',
 'Ùģ',
 'ÙĪÙĨ',
 'Ø§Øª',
 'ÙĨØ§',
 'ÙĪ',
 'Ø¹',
 'ÙĨØ§',
 'ÙĪ',
 'ÙĬÙĨ',
 'Ø§',
 'Ùģ',
 'ÙĬ',
 'ÙĤ',
 'Ø§Ø',
 '¹',
 'Ø¯',
 'Ø©',
 'Ø¨',
 'ÙĬ',
 'Ø§ÙĨ',
 'Ø§Øª',
 'Ùĥ',
 'Ùħ',
 'ØĮ',
 'Ø®',
 'Ø²',
 'ÙĨ',
 'ÙĪ',
 'Ø§',
 'Ø£',
 'ÙĬ',
 'Ø¶',
 'Ø§',
 'Ùħ',
 'ÙĪ',
 'Ø§',
 'ÙĤ',
 'Ø¹',
 'ÙĨØ§',
 'Ø§ÙĦ',
 'Ø¬',
 'Øº',
 'Ø±',
 'Ø§',
 'Ùģ',
 'ÙĬØ©',
 'ÙĪ',
 'Ø§Ø±',
 'ØŃ',
 'Ùħ',
 'ÙĪÙĨ',
 'Ø§',
 '[SEP]']

In [16]:
tokens = ["الخدمة", "سريعة", "لكن", "الأكل", "بارد"]
output = tokenizer(tokens, is_split_into_words=True)
print(output.tokens())

['[CLS]', 'Ø§ÙĦ', 'Ø®', 'Ø¯', 'Ùħ', 'Ø©', 'Ø³', 'Ø±', 'ÙĬ', 'Ø¹', 'Ø©', 'ÙĦ', 'Ùĥ', 'ÙĨ', 'Ø§ÙĦ', 'Ø£', 'Ùĥ', 'ÙĦ', 'Ø¨', 'Ø§Ø±', 'Ø¯', '[SEP]']


In [17]:
model_checkpoint = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokens = ["الخدمة", "سريعة", " Willis", "الأكل", "بارد"]
output = tokenizer(tokens, is_split_into_words=True)
print(output.tokens())

tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

['[CLS]', 'الخدمة', 'سريعة', 'W', '##ill', '##is', 'الأكل', 'بارد', '[SEP]']


In [18]:
inputs = tokenizer(data['train'][0]['tokens'], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'عمال',
 'شركات',
 'التوصيل',
 'أمثال',
 'ذبح',
 '##ونا',
 'اب',
 '##عث',
 'لي',
 'لوك',
 '##يشن',
 'في',
 'كل',
 'مرة',
 'توصيل',
 'مثلما',
 'خز',
 '##نت',
 '##م',
 'تليفونات',
 '##نا',
 'وعناوين',
 '##ا',
 'في',
 'قاعدة',
 'بيانات',
 '##كم',
 '،',
 'خز',
 '##نوا',
 'أيضا',
 'مواقع',
 '##نا',
 'الجغرافية',
 'وار',
 '##حم',
 '##ونا',
 '[SEP]']

In [19]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)
        elif word_id != previous_word_id:
            new_labels.append(labels[word_id])
        else:
            new_labels.append(labels[word_id])
        previous_word_id = word_id
    return new_labels

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=58,
        padding=True
    )
    new_labels = []
    for i, labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(i)
        aligned = align_labels_with_tokens(labels, word_ids)
        new_labels.append(aligned)
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [21]:
tokenized_datasets = data.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=data["train"].column_names,
)

Map:   0%|          | 0/3876 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/728 [00:00<?, ? examples/s]

In [22]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

2025-04-26 16:08:44.935975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745683725.120130      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745683725.173867      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [23]:
import evaluate
import numpy as np

metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [24]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

Bertmodel = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
args = TrainingArguments(
    "bert-finetuned-ner",
    eval_strategy="steps",
    eval_steps=250,
    save_strategy="epoch",
    learning_rate=3e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    per_device_train_batch_size=12,
    report_to="none",
)

In [26]:
trainer = Trainer(
    model=Bertmodel,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [27]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
250,No log,0.675218,0.660403,0.517895,0.580531,0.805953
500,0.482100,0.651733,0.663539,0.521053,0.583726,0.809156
750,0.482100,0.694756,0.620155,0.505263,0.556845,0.797664
1000,0.306200,0.740372,0.638999,0.510526,0.567583,0.799736
1250,0.306200,0.748938,0.590024,0.510526,0.547404,0.794650
1500,0.222200,0.786221,0.591667,0.523158,0.555307,0.792012


TrainOutput(global_step=1615, training_loss=0.3277279124540441, metrics={'train_runtime': 240.4245, 'train_samples_per_second': 80.607, 'train_steps_per_second': 6.717, 'total_flos': 573684754325040.0, 'train_loss': 0.3277279124540441, 'epoch': 5.0})

In [28]:
trainer.evaluate()

{'eval_loss': 0.7783383727073669,
 'eval_precision': 0.5939024390243902,
 'eval_recall': 0.5126315789473684,
 'eval_f1': 0.5502824858757063,
 'eval_accuracy': 0.7905048982667672,
 'eval_runtime': 0.9199,
 'eval_samples_per_second': 263.07,
 'eval_steps_per_second': 33.699,
 'epoch': 5.0}

In [29]:
test_dataset = tokenized_datasets["test"]

predictions_output = trainer.predict(test_dataset)
preds = predictions_output.predictions
labels = predictions_output.label_ids
metrics = predictions_output.metrics

pred_classes = np.argmax(preds, axis=1)

print("Predicted classes:", pred_classes)
print("True labels:", labels)
print("Metrics:")
print(metrics)

Predicted classes: [[ 1 11  1 ... 22 18 45]
 [ 1  2  1 ...  3  5 23]
 [ 1  2  1 ...  0  3  2]
 ...
 [ 4 33  1 ...  3 27  6]
 [28  1  3 ... 38 22 27]
 [25 19 20 ... 20 13 30]]
True labels: [[-100    2    8 ... -100 -100 -100]
 [-100    2    8 ... -100 -100 -100]
 [-100    2    8 ... -100 -100 -100]
 ...
 [-100    2    2 ... -100 -100 -100]
 [-100    8    8 ... -100 -100 -100]
 [-100    8    8 ... -100 -100 -100]]
Metrics:
{'test_loss': 0.39319923520088196, 'test_precision': 0.6766874645490641, 'test_recall': 0.6405369127516779, 'test_f1': 0.6581161219142188, 'test_accuracy': 0.8899967040210943, 'test_runtime': 2.9151, 'test_samples_per_second': 249.735, 'test_steps_per_second': 31.217}


#Task Two: Polarity NER

In [30]:
dataset_task_two = []
polarity_labels = set()

raw_data = pd.read_excel('/kaggle/input/dataset1/All_data.xlsx')

for i, (sent_id, group) in enumerate(raw_data.groupby("Sentence #")):
    tokens = group['word'].tolist()
    polarities = group['polarity'].tolist()
    polarity_labels.update(polarities)
    dataset_task_two.append({
        "id": str(i),
        "tokens": tokens,
        "ner_tags": polarities
    })

pol_label_list = sorted(polarity_labels)
pol_label2id = {label: idx for idx, label in enumerate(pol_label_list)}
id2pol_label = {idx: label for label, idx in pol_label2id.items()}

for item in dataset_task_two:
    item["ner_tags"] = [pol_label2id[tag] for tag in item["ner_tags"]]

features_pol = Features({
    "id": Value("string"),
    "tokens": Sequence(Value("string")),
    "ner_tags": Sequence(ClassLabel(names=pol_label_list))
})

random.shuffle(dataset_task_two)
n = len(dataset_task_two)
train_size = int(0.8 * n)
val_size = int(0.05 * n)

data_pol = DatasetDict({
    "train": Dataset.from_list(dataset_task_two[:train_size], features=features_pol),
    "validation": Dataset.from_list(dataset_task_two[train_size:train_size + val_size], features=features_pol),
    "test": Dataset.from_list(dataset_task_two[train_size + val_size:], features=features_pol)
})

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [31]:
pol_tokenized = data_pol.map(tokenize_and_align_labels, batched=True, remove_columns=data_pol["train"].column_names)

model_pol_ner = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2pol_label,
    label2id=pol_label2id
)

Map:   0%|          | 0/3876 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/728 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
args_pol_ner = TrainingArguments(
    "bert-ner-polarity",
    eval_strategy="steps",
    eval_steps=250,
    learning_rate=3e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    per_device_train_batch_size=12,
    save_strategy="epoch",
    report_to="none",
)

In [33]:
trainer_pol_ner = Trainer(
    model=model_pol_ner,
    args=args_pol_ner,
    train_dataset=pol_tokenized["train"],
    eval_dataset=pol_tokenized["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_31/2546147438.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_pol_ner = Trainer(


In [34]:
trainer_pol_ner.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
250,No log,0.302713,0.895749,0.895749,0.895749,0.895749
500,0.335900,0.296996,0.894907,0.894907,0.894907,0.894907
750,0.335900,0.332646,0.887610,0.887610,0.887610,0.887610
1000,0.205900,0.310790,0.898134,0.898134,0.898134,0.898134
1250,0.205900,0.334314,0.896029,0.896029,0.896029,0.896029
1500,0.129700,0.352384,0.898414,0.898414,0.898414,0.898414


TrainOutput(global_step=1615, training_loss=0.21614556563527962, metrics={'train_runtime': 240.7795, 'train_samples_per_second': 80.489, 'train_steps_per_second': 6.707, 'total_flos': 573653636401680.0, 'train_loss': 0.21614556563527962, 'epoch': 5.0})

In [35]:
trainer_pol_ner.evaluate()

{'eval_loss': 0.353480726480484,
 'eval_precision': 0.8968710537393012,
 'eval_recall': 0.8968710537393012,
 'eval_f1': 0.8968710537393012,
 'eval_accuracy': 0.8968710537393012,
 'eval_runtime': 0.9948,
 'eval_samples_per_second': 243.269,
 'eval_steps_per_second': 31.163,
 'epoch': 5.0}

In [37]:
pol_predictions, pol_labels, _ = trainer_pol_ner.predict(pol_tokenized["test"])
pol_predictions = np.argmax(pol_predictions, axis=2)

import evaluate
seqeval = evaluate.load("seqeval")

true_pol_predictions = [
    [pol_label_list[p] for (p, l) in zip(pred, label) if l != -100]
    for pred, label in zip(pol_predictions, pol_labels)
]

true_pol_labels = [
    [pol_label_list[l] for (p, l) in zip(pred, label) if l != -100]
    for pred, label in zip(pol_predictions, pol_labels)
]


results_pol = seqeval.compute(predictions=true_pol_predictions, references=true_pol_labels)

print("Test Results for Polarity Model:")
print(f"Precision: {results_pol['overall_precision']:.4f}")
print(f"Recall: {results_pol['overall_recall']:.4f}")
print(f"F1 Score: {results_pol['overall_f1']:.4f}")
print(f"Accuracy: {results_pol['overall_accuracy']:.4f}")

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Pos seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Neg seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Test Results for Polarity Model:
Precision: 0.6158
Recall: 0.6198
F1 Score: 0.6178
Accuracy: 0.8983
